In [2]:
# Dependencies to import
import pandas as pd
import scipy.stats as sts

# Cvs files to locate
california_state = "/Users/azpunit/Desktop/Project-1/Project data/csv files/California case_demographics_age.csv"
connecticut_state = "/Users/azpunit/Desktop/Project-1/Project data/csv files/Conneticut COVID-19_Cases_and_Deaths_by_Age_Group.csv"
florida_state = "/Users/azpunit/Desktop/Project-1/Project data/csv files/COVID-19 in Florida- What Is the Death Rate by Age Group.csv"


